# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [326]:
# Importing Libraries


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Input

In [327]:
ttt_df = pd.read_csv('tic-tac-toe.csv')
ttt_df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [328]:
# Assigning b to 0.5 instead of '2' or '-1' makes it so we don't need to normalize (0-1)
#  as all the values are already in the range.
ttt_df = ttt_df.replace({'x':1, 'o':0, 'b':0.5, True:1, False:0})

X = ttt_df.drop("class", axis=1)
y = ttt_df["class"]

C:\Users\Javi\AppData\Local\Temp\ipykernel_87964\907362345.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ttt_df = ttt_df.replace({'x':1, 'o':0, 'b':0.5, True:1, False:0})


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [331]:
# Splitting the dataset into the Training set and Test set (70/30)
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.3, random_state=42)

# Building the model
def seq_model():
    model = Sequential()
    model.add(Input(shape=(X.shape[1],)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer="adam", metrics=['accuracy'], loss='sparse_categorical_crossentropy')
    return model

ttt_model = seq_model()

# Training the model
ttt_model.fit(X_train, y_train, epochs=100, verbose=0)

In [332]:
# Model evaluation on test data
eval = ttt_model.evaluate(X_test, y_test)
print(f"Accuracy: {eval[1].__round__(4)}\nLoss:\t  {eval[0].__round__(4)}")

# Saving the model
ttt_model.save("tic-tac-toe.keras")

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - accuracy: 0.9396 - loss: 0.1679
Accuracy: 0.9507
Loss:	  0.1637


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [319]:
model = load_model("tic-tac-toe.keras")

y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

f1_scr = f1_score(y_test, y_pred_labels)
print(f'F1 Score: {f1_scr:.4f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
F1 Score: 0.9841


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [351]:
# Creating an instance of Adam optimizer to test changes in the learning rate
custom_adam = tf.keras.optimizers.Adam(learning_rate=0.0005) # Default = 0.001

# Splitting the dataset into the Training set and Test set (70/30)
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.3, random_state=42)

# Building the model
def seq_model():
    model = Sequential()
    model.add(Input(shape=(X.shape[1],)))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=custom_adam, metrics=['accuracy'], loss='sparse_categorical_crossentropy')
    return model

ttt_model = seq_model()

# Training the model
ttt_model.fit(X_train, y_train, epochs=100, verbose=0)

# Model evaluation on test data
eval = ttt_model.evaluate(X_test, y_test)
print(f"Accuracy: {eval[1].__round__(4)}\nLoss:\t  {eval[0].__round__(4)}")

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - accuracy: 0.9758 - loss: 0.0836
Accuracy: 0.9761
Loss:	  0.0962


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
""" 
Reducing the learning rate, adding at least one layer and increasing the epochs a bit
all seemed to improve the model.
"""